### ANALISIS Y FILTRADO INICIALES DEL DATASET 'beer_reviews.csv'


#### FUENTE DEL DATASET:

https://query.data.world/s/gib6aa6n3tmtuvrqpbdlhcbgdg7tk4


#### INFORMACION DEL DATASET:

El dataset contiene valoraciones de cervezas realizadas hasta el año 2015 por los usuarios de la web BeerAdvocates con los parametros:

		Appearance (Look) = review_appearance = APARIENCIA O IMPRESION VISUAL
		Smell = review_aroma = OLORES
		Taste = review_taste = SABOR
		Mouthfeel (Feel) = review_palate = SENSACIONES EN BOCA Y PALADAR
		Overall = review_overall = VALORACION GLOBAL

Por otro lado, como se verá después, la mayoria de las cervezas valoradas son cervezas artesanas por lo que el tipo de usuario que realiza valoraciones en BeerAdvocates es un bebedor de cerveza de cervezas artesanas aunque tambien hayan hecho valoraciones de algunas cervezas de grandes cerveceras.


#### PASOS DEL ANALISIS Y FILTRADO INICIALES:

Las pasos ejecutados para este análisis y filtrado iniciales son:

1.- Filtrado de registros de valoraciones con valores nulos.

2.- Filtrado de valoraciones repetidas para la misma cerveza por parte del mismo usuario.

3.- Limpieza de caracteres separadores en las columnas de tipo string para que unicamente quede el espacio en blanco como separador: 'beer_style', 'beer_name' 'brewery_name'.

4.- Analisis de la necesidad o no de filtrar registros de valoraciones en funcion del año en que se hicieron registrado en la columna 'review_time'.

5.- Añadido de las nuevas columnas 'review_average' y 'abv_strength' necesarias para analisis posteriores mas detallados de los parametros de las valoraciones.

		'abv_strength' = Intensidad del Grado Alcohólico, podria ser un dato para crear el perfil de usuario
		'review_average' = Media de todos los Parámetros de Valoración de una cerveza (nueva medida de valoración)


In [1]:
# Data analysis libraries
import pandas as pd
import numpy as np

In [2]:
# Leemos el fichero CSV: BEERADVOCATE RATINGS 
reviews_raw = pd.read_csv('./Data/beer_reviews.csv', delimiter=',',\
                          dtype={'brewery_name': str, 'review_profilename': str,\
                                 'beer_style': str, 'beer_name': str})

In [3]:
# Haremos una copia para trabajar con ella
dfreviews = reviews_raw.copy()

#print(type(reviews_raw))
#print(type(dfreviews))

nRow, nCol = dfreviews.shape
print('Hay',  nRow, 'filas y', nCol, 'columnas')

# Primer vistazo
dfreviews.head()

Hay 1586614 filas y 13 columnas


,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [4]:
# Estructura original del fichero
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


### FILTAREMOS REGISTROS CON ALGUN VALOR NULO

In [5]:
# Contemos los valores NULOS que hay en CADA COLUMNA
# brewery_name             15   Que haremos con estas filas ??? BORRARLAS, SON POCAS RESPECTO AL TOTAL
# review_profilename      348   Que haremos con estas filas ??? BORRARLAS AUNQUE SE PODRIA RELLENAR CON PROFILE ID SECUENCIAL GENERADO
# beer_abv              67785   Que haremos con estas filas ???
# EN PRINCIPIO ESTE PARAMETRO NO LO USAREMOS PARA RECOMENDAR ASI QUE DEJAREMOS ESTOS REGISTROS 
# SI PASAN LOS FILTROS POSTERIORES: si se tuviera que usar habra que pensar si fuera posible rellenarlos. 
print(dfreviews.isnull().sum())

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64


In [6]:
# Respecto al numero total de registros calculamos el porcentaje de registros con `beer_abv` a NULO
# 4,27% = 67785,TODAVIA QUEDARAN SUFICIENTES FILAS = 1586614
# El PARAMETRO ABV se analizara mas adelante para estimar su influencia a la hora de elegir una cerveza para comprarla
print("Porcentaje de Valores NULL de la columna `beer_abv`:", round(67785 / 1586614 * 100, 2),"%")

Porcentaje de Valores NULL de la columna `beer_abv`: 4.27 %


In [7]:
# BORRAR TODAS LAS FILAS CON ALGUN VALOR NULO PORQUE NO SON UN PORCENTAJE ALTO RESPECTO AL TOTAL
dfreviews = dfreviews.dropna()
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518478 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1518478 non-null int64
brewery_name          1518478 non-null object
review_time           1518478 non-null int64
review_overall        1518478 non-null float64
review_aroma          1518478 non-null float64
review_appearance     1518478 non-null float64
review_profilename    1518478 non-null object
beer_style            1518478 non-null object
review_palate         1518478 non-null float64
review_taste          1518478 non-null float64
beer_name             1518478 non-null object
beer_abv              1518478 non-null float64
beer_beerid           1518478 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 162.2+ MB


In [8]:
print(dfreviews.isnull().sum())

brewery_id            0
brewery_name          0
review_time           0
review_overall        0
review_aroma          0
review_appearance     0
review_profilename    0
beer_style            0
review_palate         0
review_taste          0
beer_name             0
beer_abv              0
beer_beerid           0
dtype: int64


In [9]:
# BORRADO HA SIDO CORRECTO ... 68136 = 15 + 348 + 67785
print('Se han borrado', reviews_raw.shape[0] - dfreviews.shape[0], 'filas')
reviews_raw.shape[0], dfreviews.shape[0]

Se han borrado 68136 filas


(1586614, 1518478)

## FILTRAREMOS USUARIOS QUE HAN VOTADO MAS DE UNA VEZ A LA MISMA CERVEZA:

FILTRAREMOS Y SOLO NOS QUEDAREMOS 1 VALORACION: LA QUE TENGA PUNTUACION MAS ALTA DE 'review_overall'

In [10]:
# EXISTEN USUARIOS QUE HAN VOTADO MAS DE UNA VEZ A LA MISMA CERVEZA:
# FILTRAREMOS Y SOLO NOS QUEDAREMOS 1 VALORACION = LA MAS ALTA DE 'review_overall'
dfreviews.sort_values(['review_profilename', 'beer_name','review_overall'], ascending=False).head(40)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1373610,721,Carlsberg Danmark A/S,1258125902,5.0,5.0,5.0,zzajjber,Euro Pale Lager,5.0,5.0,Carlsberg Beer,5.00,2270
873579,147,Stone Brewing Co.,1147945805,4.0,4.5,4.0,zyzygy,American Barleywine,4.5,4.5,Stone Old Guardian Barley Wine Style Ale 2006,11.20,28687
293561,35,Boston Beer Company (Samuel Adams),1141676810,4.0,3.5,3.5,zyzygy,English Brown Ale,3.0,3.5,Samuel Adams Brown Ale,5.35,27514
773245,42,Brewery Ommegang,1147935912,4.5,4.5,5.0,zyzygy,Saison / Farmhouse Ale,4.5,5.0,Hennepin (Farmhouse Saison),7.70,141
173551,94,Long Trail Brewing Co.,1139008381,4.5,4.0,4.0,zyzygy,Altbier,4.0,4.5,Double Bag,7.20,273
566468,33,Berkshire Brewing Company Inc.,1141339493,4.0,4.0,3.5,zyzygy,Russian Imperial Stout,3.5,4.5,Berkshire Russian Imperial Stout,8.50,1112
366706,22,Unibroue,1241402519,3.0,2.0,3.0,zythus,Fruit / Vegetable Beer,4.0,3.0,Éphémère (Apple),5.50,3523
1486864,23980,Yuengling Brewery,1267120909,4.5,4.0,4.0,zythus,Bock,4.5,4.0,Yuengling Bock,5.10,47812
71905,140,Sierra Nevada Brewing Co.,1237658573,4.0,3.0,3.5,zythus,Extra Special / Strong Bitter (ESB),5.0,4.5,Sierra Nevada ESB (Early Spring Beer),5.90,40492
521081,113,Samuel Smith Old Brewery (Tadcaster),1237681606,5.0,4.5,4.5,zythus,English Porter,5.0,5.0,"Samuel Smith's, The Famous Taddy Porter",5.00,572


In [11]:
# EJEMPLO: EL USUARIO 'zythus' HA VOTADO 3 VECES LA CERVEZA 'Samuel Smith Old Brewery (Tadcaster)' 
dfreviews[dfreviews['review_profilename'] == 'zythus'].\
    sort_values(['review_profilename', 'beer_name','review_overall'], ascending=False)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
366706,22,Unibroue,1241402519,3.0,2.0,3.0,zythus,Fruit / Vegetable Beer,4.0,3.0,Éphémère (Apple),5.5,3523
1486864,23980,Yuengling Brewery,1267120909,4.5,4.0,4.0,zythus,Bock,4.5,4.0,Yuengling Bock,5.1,47812
71905,140,Sierra Nevada Brewing Co.,1237658573,4.0,3.0,3.5,zythus,Extra Special / Strong Bitter (ESB),5.0,4.5,Sierra Nevada ESB (Early Spring Beer),5.9,40492
521081,113,Samuel Smith Old Brewery (Tadcaster),1237681606,5.0,4.5,4.5,zythus,English Porter,5.0,5.0,"Samuel Smith's, The Famous Taddy Porter",5.0,572
520548,113,Samuel Smith Old Brewery (Tadcaster),1242257696,5.0,5.0,5.0,zythus,Euro Pale Lager,5.0,5.0,Samuel Smith's Pure Brewed Lager Beer,5.0,574
516121,113,Samuel Smith Old Brewery (Tadcaster),1243812668,5.0,4.0,4.5,zythus,English Brown Ale,5.0,4.5,Samuel Smith's Nut Brown Ale,5.0,576
512078,428,Mahrs-Bräu,1237678432,2.5,4.0,3.0,zythus,Keller Bier / Zwickel Bier,3.0,2.0,Mahr's Ungespundet-hefetrüb,5.2,2412
385043,184,Bierbrouwerij De Koningshoeven B.V.,1237679589,4.5,4.5,4.0,zythus,Dubbel,4.5,4.5,La Trappe Dubbel (Koningshoeven / Dominus),7.0,626


In [12]:
# CLASIFICAMOS POR USUARIO, CERVEZA Y 'review_overall'
dfreviews = dfreviews.sort_values(['review_profilename', 'beer_name','review_overall'], ascending=False)

# PARA CADA USUARIO, MANTENDREMOS LA VOTACION DE MAYOR VALOR DE 'review_overall' Y BORRAREMOS LAS DEMAS
dfreviews = dfreviews.drop_duplicates(subset= ['review_profilename','beer_name'], keep='first')

In [13]:
# RESULTADO FINAL...

# PORCENTAJE DE REGISTROS QUE TENIAN VOTACIONES REPETIDAS DE MISMO USUARIO Y MISMA CERVEZA
# ANTES DE BORRARLOS TENIA 1518478 REGISTROS
print("Porcentaje de Votaciones Repetidas:", round((1518478 - 1496263)/ 1518478 * 100, 2),"%")

# SOLO HE BORRADO UN 1,46% Y SIGUEN QUEDANDO 1496263 QUE SON BASTANTES VALORACIONES
dfreviews.info()

Porcentaje de Votaciones Repetidas: 1.46 %
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1496263 entries, 1373610 to 803466
Data columns (total 13 columns):
brewery_id            1496263 non-null int64
brewery_name          1496263 non-null object
review_time           1496263 non-null int64
review_overall        1496263 non-null float64
review_aroma          1496263 non-null float64
review_appearance     1496263 non-null float64
review_profilename    1496263 non-null object
beer_style            1496263 non-null object
review_palate         1496263 non-null float64
review_taste          1496263 non-null float64
beer_name             1496263 non-null object
beer_abv              1496263 non-null float64
beer_beerid           1496263 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 159.8+ MB


In [14]:
# Los PARAMETROS DE VALORACION DE CERVEZAS tienen una PUNTUACION ENTRE 1 Y 5, CADA UNO
# review_overall tiene MINIMO de 0 ES INVALIDO: LOS QUITAREMOS MAS ABAJO
# review_appearance tiene MINIMO de 0 ES INVALIDO: LOS QUITAREMOS MAS ABAJO
# beer_abv tiene MINIMO MENOR QUE 0 ES INVALIDO: TODAVIA NO SE SI LO USAREMOS, LO DEJAMOS ASI
dfreviews.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.496263e+06,1.496263e+06,1.496263e+06,1.496263e+06,1.496263e+06,1.496263e+06,1.496263e+06,1.496263e+06,1.496263e+06
mean,3.068176e+03,1.225562e+09,3.825543e+00,3.746441e+00,3.850315e+00,3.754201e+00,3.804601e+00,7.046094e+00,2.144066e+04
std,5.541302e+03,7.574088e+07,7.176279e-01,6.953771e-01,6.144186e-01,6.793107e-01,7.287044e-01,2.325745e+00,2.180017e+04
min,1.000000e+00,8.843904e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,5.000000e+00
25%,1.410000e+02,1.175888e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.655000e+03
50%,4.130000e+02,1.240556e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.291800e+04
75%,2.250000e+03,1.289239e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.928200e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731600e+04


##### Scoring All Ones or Fives
Rating a beer with all ones (no redeeming qualities) or fives (no room for improvement) should be rare. It can be a sign of someone abusing the rating system, too, so consider writing a review to back up your unusually low or high rating.

##### POR LO TANTO LAS PUNTUACIONES TOTALES INUSUALMENTE BAJAS O ALTAS SE DEBERIAN QUITAR PORQUE NO SE CONSIDERAN CORRECTAS.

brewery_id 	review_overall 	review_aroma 	review_appearance 	review_palate 	review_taste 	beer_abv 	beer_beerid

min 	1.000000e+00 	0.000000e+00 	1.000000e+00 	0.000000e+00 	1.000000e+00 	1.000000e+00 	1.000000e-02 	3.000000e+00

max 	2.800300e+04 	5.000000e+00 	5.000000e+00 	5.000000e+00 	5.000000e+00 	5.000000e+00 	5.770000e+01 	7.731700e+04

LAS RATINGS CON PUNTUACIONES >= 5 SE DEBERIAN QUITAR PORQUE NO SE CONSIDERAN CORRECTAS

In [15]:
# QUITAREMOS:
# review_overall < 1 (0 ES INVALIDO)
# review_appearance < 1 (0 ES INVALIDO) y,
# SE HAN CORREGIDO LOS VALORES DE:
# 'review_overall', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste'
dfreviews = dfreviews[(dfreviews['review_overall'] >= 1) | \
                      (dfreviews['review_appearance'] >= 1)]

dfreviews.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06
mean,3.068184e+03,1.225563e+09,3.825561e+00,3.746447e+00,3.850333e+00,3.754206e+00,3.804609e+00,7.046109e+00,2.144074e+04
std,5.541313e+03,7.573988e+07,7.175819e-01,6.953723e-01,6.143636e-01,6.793064e-01,7.286961e-01,2.325741e+00,2.180019e+04
min,1.000000e+00,8.843904e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,5.000000e+00
25%,1.410000e+02,1.175890e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.655000e+03
50%,4.130000e+02,1.240557e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.292000e+04
75%,2.250000e+03,1.289239e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.928300e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731600e+04


In [16]:
# PORCENTAJE DE REGISTROS QUE TENIAN VOTACIONES INCORRECTAS DE 'review_overall' Y 'review_appearance'
# ANTES DE BORRARLOS TENIA 1496263 REGISTROS
print("Porcentaje de Votaciones 0:", round((1496263 - 1496256)/ 1496263 * 100, 4),"%")
print("Numero de registros con Votaciones 0:", (1496263 - 1496256))
dfreviews.head()

Porcentaje de Votaciones 0: 0.0005 %
Numero de registros con Votaciones 0: 7


,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1373610,721,Carlsberg Danmark A/S,1258125902,5.0,5.0,5.0,zzajjber,Euro Pale Lager,5.0,5.0,Carlsberg Beer,5.00,2270
873579,147,Stone Brewing Co.,1147945805,4.0,4.5,4.0,zyzygy,American Barleywine,4.5,4.5,Stone Old Guardian Barley Wine Style Ale 2006,11.20,28687
293561,35,Boston Beer Company (Samuel Adams),1141676810,4.0,3.5,3.5,zyzygy,English Brown Ale,3.0,3.5,Samuel Adams Brown Ale,5.35,27514
773245,42,Brewery Ommegang,1147935912,4.5,4.5,5.0,zyzygy,Saison / Farmhouse Ale,4.5,5.0,Hennepin (Farmhouse Saison),7.70,141
173551,94,Long Trail Brewing Co.,1139008381,4.5,4.0,4.0,zyzygy,Altbier,4.0,4.5,Double Bag,7.20,273


In [17]:
# QUITAREMOS LAS PUNTUACIONES INCORRECTAS TIPO "TODO 1" Y TIPO "TODO 5" 
# 'review_overall', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste'
# EJEMPLO: USUARIO 'zzajjber' PUNTUA TODO 5 A CERVEZA 'Carlsberg Danmark A/S'
dfreviews[(dfreviews['review_overall'] == 5) & \
          (dfreviews['review_aroma'] == 5) & \
          (dfreviews['review_appearance'] == 5) & \
          (dfreviews['review_palate'] == 5) & \
          (dfreviews['review_taste'] == 5)].head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1373610,721,Carlsberg Danmark A/S,1258125902,5.0,5.0,5.0,zzajjber,Euro Pale Lager,5.0,5.0,Carlsberg Beer,5.0,2270
520548,113,Samuel Smith Old Brewery (Tadcaster),1242257696,5.0,5.0,5.0,zythus,Euro Pale Lager,5.0,5.0,Samuel Smith's Pure Brewed Lager Beer,5.0,574
1437092,63,Deschutes Brewery,1218858317,5.0,5.0,5.0,zymrgy,American Porter,5.0,5.0,Black Butte XX,11.0,42836
1491593,313,Brouwerij Westvleteren (Sint-Sixtusabdij van W...,1230528577,5.0,5.0,5.0,zuggy9,Quadrupel (Quad),5.0,5.0,Trappist Westvleteren 12,10.2,1545
880396,147,Stone Brewing Co.,1168974255,5.0,5.0,5.0,zuggy9,Russian Imperial Stout,5.0,5.0,Stone Imperial Russian Stout,10.5,1160


In [18]:
# EJEMPLO: USUARIO 'zeff80' PUNTUA TODO 5 A CERVEZA 'Chili Beer Co.'
dfreviews[(dfreviews['review_overall'] == 1) & \
          (dfreviews['review_aroma'] == 1) & \
          (dfreviews['review_appearance'] == 1) & \
          (dfreviews['review_palate'] == 1) & \
          (dfreviews['review_taste'] == 1)].head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1074612,677,Chili Beer Co.,1178150667,1.0,1.0,1.0,zeff80,Chile Beer,1.0,1.0,Crazy Ed's Cave Creek Chili Beer,4.2,2213
609925,106,JOS. Schlitz Brewing Co. (Pabst),1312817019,1.0,1.0,1.0,yuenglingfan101,American Adjunct Lager,1.0,1.0,Schlitz,4.6,263
1299151,29,Anheuser-Busch,1278607852,1.0,1.0,1.0,yuenglingfan101,Light Lager,1.0,1.0,Natural Light,4.2,1524
53815,81,Heineken Nederland B.V.,1278606194,1.0,1.0,1.0,yuenglingfan101,Euro Pale Lager,1.0,1.0,Heineken Lager Beer,5.0,246
118302,302,Genesee Brewing Co. / Dundee Brewing Co.,1278606481,1.0,1.0,1.0,yuenglingfan101,American IPA,1.0,1.0,Dundee India Pale Ale,6.3,25144


In [19]:
# 5771 REGISTROS CON PUNTUACIONES INCORRECTAS TIPO "TODO 5"
dfreviews[(dfreviews['review_overall'] == 5) & \
          (dfreviews['review_aroma'] == 5) & \
          (dfreviews['review_appearance'] == 5) & \
          (dfreviews['review_palate'] == 5) & \
          (dfreviews['review_taste'] == 5)].count()

brewery_id            5771
brewery_name          5771
review_time           5771
review_overall        5771
review_aroma          5771
review_appearance     5771
review_profilename    5771
beer_style            5771
review_palate         5771
review_taste          5771
beer_name             5771
beer_abv              5771
beer_beerid           5771
dtype: int64

In [20]:
# 975 REGISTROS CON PUNTUACIONES INCORRECTAS TIPO "TODO 1"
dfreviews[(dfreviews['review_overall'] == 1) & \
          (dfreviews['review_aroma'] == 1) & \
          (dfreviews['review_appearance'] == 1) & \
          (dfreviews['review_palate'] == 1) & \
          (dfreviews['review_taste'] == 1)].count()

brewery_id            975
brewery_name          975
review_time           975
review_overall        975
review_aroma          975
review_appearance     975
review_profilename    975
beer_style            975
review_palate         975
review_taste          975
beer_name             975
beer_abv              975
beer_beerid           975
dtype: int64

In [21]:
dfreviews = dfreviews.drop(dfreviews[((dfreviews['review_overall'] == 1) & \
                                      (dfreviews['review_aroma'] == 1) & \
                                      (dfreviews['review_appearance'] == 1) & \
                                      (dfreviews['review_palate'] == 1) & \
                                      (dfreviews['review_taste'] == 1)) | \
                                     ((dfreviews['review_overall'] == 5) & \
                                      (dfreviews['review_aroma'] == 5) & \
                                      (dfreviews['review_appearance'] == 5) & \
                                      (dfreviews['review_palate'] == 5) & \
                                      (dfreviews['review_taste'] == 5))].index).copy()
dfreviews.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
873579,147,Stone Brewing Co.,1147945805,4.0,4.5,4.0,zyzygy,American Barleywine,4.5,4.5,Stone Old Guardian Barley Wine Style Ale 2006,11.20,28687
293561,35,Boston Beer Company (Samuel Adams),1141676810,4.0,3.5,3.5,zyzygy,English Brown Ale,3.0,3.5,Samuel Adams Brown Ale,5.35,27514
773245,42,Brewery Ommegang,1147935912,4.5,4.5,5.0,zyzygy,Saison / Farmhouse Ale,4.5,5.0,Hennepin (Farmhouse Saison),7.70,141
173551,94,Long Trail Brewing Co.,1139008381,4.5,4.0,4.0,zyzygy,Altbier,4.0,4.5,Double Bag,7.20,273
566468,33,Berkshire Brewing Company Inc.,1141339493,4.0,4.0,3.5,zyzygy,Russian Imperial Stout,3.5,4.5,Berkshire Russian Imperial Stout,8.50,1112


In [22]:
# PORCENTAJE DE REGISTROS QUE TENIAN VOTACIONES INCORRECTAS DE 'review_overall' Y 'review_appearance'
# ANTES DE BORRARLOS TENIA 1496256 REGISTROS, SE BORRARON 6746,
x = dfreviews.shape[0]
print("Porcentaje de Votaciones (TODO 1 o TODO 5):", round((1496256 - x)/ 1496256 * 100, 4),"%")
print("Numero de registros con Votaciones (TODO 1 o TODO 5):", (1496256 - x))
print("Numero de registros de Votaciones ACTUAL:", x)

Porcentaje de Votaciones (TODO 1 o TODO 5): 0.4509 %
Numero de registros con Votaciones (TODO 1 o TODO 5): 6746
Numero de registros de Votaciones ACTUAL: 1489510


In [23]:
# Los PARAMETROS DE VALORACION DE CERVEZAS AHORA SI TIENEN SU PUNTUACION ENTRE 1 Y 5, CADA UNO
# beer_abv tiene MINIMO MENOR QUE 0 SIGUE SIENDO INVALIDO: 
# NO BORRAREMOS ESAS VALORACIONES PORQUE TIENE VALORACIONES VALIDAS
dfreviews.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06
mean,3.071700e+03,1.225595e+09,3.822860e+00,3.743388e+00,3.847745e+00,3.751183e+00,3.801813e+00,7.041184e+00,2.145860e+04
std,5.543628e+03,7.568307e+07,7.118135e-01,6.889830e-01,6.072120e-01,6.727253e-01,7.229868e-01,2.319915e+00,2.180543e+04
min,1.000000e+00,8.843904e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,5.000000e+00
25%,1.410000e+02,1.175913e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.655000e+03
50%,4.150000e+02,1.240588e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.299300e+04
75%,2.295000e+03,1.289221e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.929600e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731600e+04


### LIMPIAREMOS LOS SEPARADORES DE LOS VALORES STRING

##### Beer Style, Beer Name and Brewery Name Values Fixing

Necesitamos tenerlos sin separadores para:

1.-Los análisis de parámetros en pasos posteriores.

2.- El mapeo de los estilos de cerveza registrados en el dataset 'beer_reviews.csv' y los existentes en la web de BeerAdvocates para 2019 cuyo objetivo es poder tener datos adicionales de los estilos de cervezas para mostrar con las cervezas finalmente recomendadas.


In [24]:
# DIVIDIR DESPUES DE SEPARADOR (' / ' o ' - ' ) MANTENER SOLO LA CADENA ANTERIOR
# Ejemplo: brewery_name = 'Rusty Truck Brewing Company / Roadhouse 101'
dfreviews['brewery_name'] = dfreviews['brewery_name'].str.split(' / ').str[0]
dfreviews['brewery_name'] = dfreviews['brewery_name'].str.split(' - ').str[0]

In [25]:
# SOLO FILTRAR EL SEPARADOR (' / ' o ' - ' ) Y SUSTITUIRLO POR UN BLANCO ' '
# Ejemplo: beer_name = 'Barrel Aged Port Royal Stout W/ Vanilla Beans'
dfreviews['beer_name'] = dfreviews['beer_name'].str.replace(' / ', ' ')
dfreviews['beer_name'] = dfreviews['beer_name'].str.replace(' - ', ' ')

In [26]:
# SOLO FILTRAR EL SEPARADOR (' / ' o ' - ' ) Y SUSTITUIRLO POR UN BLANCO ' '
# Ejemplo: beer_style = 'Foreign / Export Stout'
dfreviews['beer_style'] = dfreviews['beer_style'].str.replace(' / ', ' ')
dfreviews['beer_style'] = dfreviews['beer_style'].str.replace(' - ', ' ')

In [27]:
dfreviews[['brewery_name', 'beer_name','beer_style']].head()

,brewery_name,beer_name,beer_style
873579,Stone Brewing Co.,Stone Old Guardian Barley Wine Style Ale 2006,American Barleywine
293561,Boston Beer Company (Samuel Adams),Samuel Adams Brown Ale,English Brown Ale
773245,Brewery Ommegang,Hennepin (Farmhouse Saison),Saison Farmhouse Ale
173551,Long Trail Brewing Co.,Double Bag,Altbier
566468,Berkshire Brewing Company Inc.,Berkshire Russian Imperial Stout,Russian Imperial Stout


In [28]:
dfreviews[['brewery_name', 'beer_name','beer_style']].tail()

,brewery_name,beer_name,beer_style
1041524,Hair of the Dog Brewing Company,Adam,Old Ale
1173937,Dogfish Head Brewery,90 Minute IPA,American Double Imperial IPA
1517475,21st Amendment Brewery,21st Amendment IPA,American IPA
224287,Great Divide Brewing Company,15th Anniversary Wood Aged,American Double Imperial IPA
803466,The Lost Abbey,10 Commandments,Belgian Strong Dark Ale


In [29]:
dfreviews.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
873579,147,Stone Brewing Co.,1147945805,4.0,4.5,4.0,zyzygy,American Barleywine,4.5,4.5,Stone Old Guardian Barley Wine Style Ale 2006,11.20,28687
293561,35,Boston Beer Company (Samuel Adams),1141676810,4.0,3.5,3.5,zyzygy,English Brown Ale,3.0,3.5,Samuel Adams Brown Ale,5.35,27514
773245,42,Brewery Ommegang,1147935912,4.5,4.5,5.0,zyzygy,Saison Farmhouse Ale,4.5,5.0,Hennepin (Farmhouse Saison),7.70,141
173551,94,Long Trail Brewing Co.,1139008381,4.5,4.0,4.0,zyzygy,Altbier,4.0,4.5,Double Bag,7.20,273
566468,33,Berkshire Brewing Company Inc.,1141339493,4.0,4.0,3.5,zyzygy,Russian Imperial Stout,3.5,4.5,Berkshire Russian Imperial Stout,8.50,1112


### ANALIZAREMOS LA VARIABLE 'review_time' CONTRA EL NUMERO DE VALORACIONES POR AÑO

Hay dos objetivos:

1.- Limpiar valoraciones de años que no aporten mucha informacion

2.- Ver la evolucion en el tiempo del numero de valoraciones realizadas por los usuarios


In [30]:
# Convertimos los valores de la columna 'review_time' al formato adecuado de FECHA Y HORA
# datetime64 = 2009-02-16 20:57:03
dfreviews['review_time'] = pd.to_datetime(dfreviews['review_time'], unit = 's' )

In [31]:
# Numero de Valoraciones por cada dia
dfreviews['review_time'].map(lambda x: x.strftime('%Y-%m-%d')).head()

873579    2006-05-18
293561    2006-03-06
773245    2006-05-18
173551    2006-02-03
566468    2006-03-02
Name: review_time, dtype: object

In [32]:
# Parece que hay pocas valoraciones hasta el año 2000, no voy a quitarlas porque no perjudicarian a recomendaciones
# count() Cuenta lo valores contables ignorando las filas con los nulos
dfreviews.groupby(dfreviews['review_time'].map(lambda x: x.strftime('%Y')))['review_overall'].count().to_frame()

,review_overall
review_time,
1998,71
1999,31
2000,92
2001,1619
2002,19986
2003,50583
2004,63238
2005,81771
2006,119942


## AÑADIREMOS OTRAS VARIABLES NECESARIAS PARA PASOS POSTERIORES

### 'review_average' = BASELINE DE MEDIDA DE VALORACION GLOBAL PARA ANALISIS 

Añadiremos una NUEVA COLUMNA 'review_average' QUE SERA LA MEDIA DE TODOS LOS PARAMETROS DE VALORACION DE LAS CERVEZAS.

AL MISMO TIEMPO, TIEMPO NOS VALDRÁ COMO OTRA MEDIDA DE LA CALIDAD DE LA CERVEZA.


In [33]:
# SE HA MOVIDO DESDE EL PASO POSTERIOR "02-Users_And_Beer_Ratings_Analysis.ipynb"
dfreviews['review_average'] = dfreviews.apply( lambda row:\
                    (row['review_overall'] + row['review_aroma'] +\
                     row['review_appearance'] + row['review_palate'] +\
                     row['review_taste'] ) / 5, 
                     axis=1)

In [34]:
dfreviews.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,review_average
873579,147,Stone Brewing Co.,2006-05-18 09:50:05,4.0,4.5,4.0,zyzygy,American Barleywine,4.5,4.5,Stone Old Guardian Barley Wine Style Ale 2006,11.20,28687,4.3
293561,35,Boston Beer Company (Samuel Adams),2006-03-06 20:26:50,4.0,3.5,3.5,zyzygy,English Brown Ale,3.0,3.5,Samuel Adams Brown Ale,5.35,27514,3.5
773245,42,Brewery Ommegang,2006-05-18 07:05:12,4.5,4.5,5.0,zyzygy,Saison Farmhouse Ale,4.5,5.0,Hennepin (Farmhouse Saison),7.70,141,4.7
173551,94,Long Trail Brewing Co.,2006-02-03 23:13:01,4.5,4.0,4.0,zyzygy,Altbier,4.0,4.5,Double Bag,7.20,273,4.2
566468,33,Berkshire Brewing Company Inc.,2006-03-02 22:44:53,4.0,4.0,3.5,zyzygy,Russian Imperial Stout,3.5,4.5,Berkshire Russian Imperial Stout,8.50,1112,3.9


In [35]:
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1489510 entries, 873579 to 803466
Data columns (total 14 columns):
brewery_id            1489510 non-null int64
brewery_name          1489510 non-null object
review_time           1489510 non-null datetime64[ns]
review_overall        1489510 non-null float64
review_aroma          1489510 non-null float64
review_appearance     1489510 non-null float64
review_profilename    1489510 non-null object
beer_style            1489510 non-null object
review_palate         1489510 non-null float64
review_taste          1489510 non-null float64
beer_name             1489510 non-null object
beer_abv              1489510 non-null float64
beer_beerid           1489510 non-null int64
review_average        1489510 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(2), object(4)
memory usage: 170.5+ MB


### 'abv_strength' = INTENSIDAD DE ABV PARA ANALISIS 

Añadiremos una NUEVA COLUMNA 'abv_strength' cuyo valor sera dado por la tabla descrita a continuacion.

Se usara para el analisis de los pasos posteriores y tambien podra ser usado para obtener informacion del perfil del usuario.


#### RECORDATORIO SOBRE EL GRADO ALCOHOLICO ABV

Segun http://dev.bjcp.org/beer-styles/introduction-to-beer-styles

Low Strength = intensidad-baja:	<4% ABV

Medium Strength = intensidad-estandar: 4-6% ABV

High Strength = intensidad-alta: 6-9% ABV

Strong Strength = intensidad-muy-alta: >9% ABV

In [42]:
# MOVIDO DESDE "03-Beer_Parameters_Analysis.ipynb"
def abv_strength(abv_val):
    if (abv_val >= 1.0 and abv_val < 4.0):
        return 1
    elif (abv_val >= 4.0 and abv_val < 6.0):
        return 2
    elif (abv_val >= 6.0 and abv_val < 9.0):
        return 3
    else:
        return 4

In [47]:
dfreviews['abv_strength'] = dfreviews.apply(lambda row: \
                                            abv_strength(row['beer_abv']),
                                            axis=1)

In [48]:
dfreviews.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,review_average,abv_strength
873579,147,Stone Brewing Co.,2006-05-18 09:50:05,4.0,4.5,4.0,zyzygy,American Barleywine,4.5,4.5,Stone Old Guardian Barley Wine Style Ale 2006,11.20,28687,4.3,4
293561,35,Boston Beer Company (Samuel Adams),2006-03-06 20:26:50,4.0,3.5,3.5,zyzygy,English Brown Ale,3.0,3.5,Samuel Adams Brown Ale,5.35,27514,3.5,2
773245,42,Brewery Ommegang,2006-05-18 07:05:12,4.5,4.5,5.0,zyzygy,Saison Farmhouse Ale,4.5,5.0,Hennepin (Farmhouse Saison),7.70,141,4.7,3
173551,94,Long Trail Brewing Co.,2006-02-03 23:13:01,4.5,4.0,4.0,zyzygy,Altbier,4.0,4.5,Double Bag,7.20,273,4.2,3
566468,33,Berkshire Brewing Company Inc.,2006-03-02 22:44:53,4.0,4.0,3.5,zyzygy,Russian Imperial Stout,3.5,4.5,Berkshire Russian Imperial Stout,8.50,1112,3.9,3


In [49]:
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1489510 entries, 873579 to 803466
Data columns (total 15 columns):
brewery_id            1489510 non-null int64
brewery_name          1489510 non-null object
review_time           1489510 non-null datetime64[ns]
review_overall        1489510 non-null float64
review_aroma          1489510 non-null float64
review_appearance     1489510 non-null float64
review_profilename    1489510 non-null object
beer_style            1489510 non-null object
review_palate         1489510 non-null float64
review_taste          1489510 non-null float64
beer_name             1489510 non-null object
beer_abv              1489510 non-null float64
beer_beerid           1489510 non-null int64
review_average        1489510 non-null float64
abv_strength          1489510 non-null int64
dtypes: datetime64[ns](1), float64(7), int64(3), object(4)
memory usage: 181.8+ MB


In [50]:
dfreviews.describe()

,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,review_average,abv_strength
count,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06,1.489510e+06
mean,3.071700e+03,3.822860e+00,3.743388e+00,3.847745e+00,3.751183e+00,3.801813e+00,7.041184e+00,2.145860e+04,3.793398e+00,2.805335e+00
std,5.543628e+03,7.118135e-01,6.889830e-01,6.072120e-01,6.727253e-01,7.229868e-01,2.319915e+00,2.180543e+04,5.712579e-01,7.839940e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,5.000000e+00,1.100000e+00,1.000000e+00
25%,1.410000e+02,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.655000e+03,3.500000e+00,2.000000e+00
50%,4.150000e+02,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.299300e+04,3.900000e+00,3.000000e+00
75%,2.295000e+03,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.929600e+04,4.200000e+00,3.000000e+00
max,2.800300e+04,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731600e+04,4.900000e+00,4.000000e+00


## FINALMENTE, GUARDAREMOS ESTE NUEVO DATASET PARA EL SIGUIENTE PASO DE ANALISIS

In [51]:
# almacenaremos el DATASET LIMPIADO DE NULLs, etc
dfreviews.to_csv('./Data/beer_reviews_v1.csv', sep=',', index=False)